## Matrix shape for MLP

- MLP has no idea of time step. So there is no separate dimension in the matrix for time step
- MLP just care about features - the columns
- MLPs require that the shape of the input portion of each sample is a vector. With a multivariate input, we will have multiple vectors, one for each time step. We can flatten the temporal structure of each input sample,

# Univariate MLP Models

In [24]:
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

def split_sequence(series, n_step):
    pred_step = 1
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-pred_step-n_step+1):
        x_list.append(series[s:s+n_step])
        y_list.append(series[s+n_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [2]:
data = np.arange(10, 100, 10)
data

array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [3]:
n_step = 3
X, y = split_sequence(data, n_step)
X.shape

(6, 3)

In [4]:
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
[40 50 60 70 80 90]


In [5]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_step))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [7]:
# demonstrate a prediction
x_test = np.array([70, 80, 90])
x_test_resh = x_test.reshape(1, n_step)
x_test_resh

array([[70, 80, 90]])

In [8]:
yhat = model.predict(x_test_resh)
yhat

array([[100.08584]], dtype=float32)

## Multivariate MLP - Multiple Input Series

In [9]:
def split_sequence(series, n_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-n_step+1):
        x_list.append(series[s:s+n_step, :-1])
        y_list.append(series[s+n_step-1, -1])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [10]:
in_seq1 = np.arange(10, 100, 10)
in_seq2 = np.arange(15, 105, 10)
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(in_seq1.shape[0])])
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)
print(in_seq1.shape, in_seq2.shape, out_seq.shape)
print(in_seq1, in_seq2, out_seq)

(9, 1) (9, 1) (9, 1)
[[10]
 [20]
 [30]
 [40]
 [50]
 [60]
 [70]
 [80]
 [90]] [[15]
 [25]
 [35]
 [45]
 [55]
 [65]
 [75]
 [85]
 [95]] [[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [11]:
# horizontally stack columns
dataset  = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset.shape)
print(dataset)

(9, 3)
[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [12]:
n_steps = 3
X, y = split_sequence(dataset, n_steps)
print(X.shape, y.shape)
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


#### We can flatten the temporal structure of each input sample, so that 

In [13]:
print(X.shape)

(7, 3, 2)


In [14]:
resulting_feature = X.shape[1] * X.shape[2]
X_flat = X.reshape(X.shape[0], resulting_feature)
print(X_flat)

[[10 15 20 25 30 35]
 [20 25 30 35 40 45]
 [30 35 40 45 50 55]
 [40 45 50 55 60 65]
 [50 55 60 65 70 75]
 [60 65 70 75 80 85]
 [70 75 80 85 90 95]]


In [15]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=resulting_feature))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [16]:
model.fit(X_flat, y, epochs=2000, verbose=0)

In [17]:
### Test data
x_test_raw = np.array([[80, 85], [90, 95], [100, 105]])
# x_test_raw = np.array([[8, 5], [9, 5], [100, 105]])
print(x_test_raw)
#
x_test = x_test_raw.reshape(1, resulting_feature)
print(x_test)

[[ 80  85]
 [ 90  95]
 [100 105]]
[[ 80  85  90  95 100 105]]


In [18]:
yhat = model.predict(x_test)
print(yhat)

[[205.57687]]


## Multi-headed MLP Model

In [34]:
n_step = 3

# first input model
visible1 = Input(shape=(n_step,))
dense1 = Dense(100, activation='relu')(visible1)
# second input model
visible2 = Input(shape=(n_step,))
dense2 = Dense(100, activation='relu')(visible2)
# merge input model
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

In [28]:
print(X[:3])
print(X.shape)

[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]]
(7, 3, 2)


In [32]:
X1 = X[:, :, 0]
X2 = X[:, :, 1]
print(X1[:3])
print(X1.shape)
print(X2[:3])
print(X2.shape)
print([X1, X2])

[[10 20 30]
 [20 30 40]
 [30 40 50]]
(7, 3)
[[15 25 35]
 [25 35 45]
 [35 45 55]]
(7, 3)
[array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70],
       [60, 70, 80],
       [70, 80, 90]]), array([[15, 25, 35],
       [25, 35, 45],
       [35, 45, 55],
       [45, 55, 65],
       [55, 65, 75],
       [65, 75, 85],
       [75, 85, 95]])]


In [35]:
model.fit([X1, X2], epochs=2000, verbose=0)

IndexError: list index out of range